In [ ]:
!pip install anomalib
!pip install lightning

In [ ]:
import torch
torch.set_float32_matmul_precision("medium")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
import numpy as np
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from matplotlib import pyplot as plt
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from anomalib.data import PredictDataset, MVTec, Visa
from anomalib.engine import Engine
from anomalib.models import Fastflow, EfficientAd, Cflow
from anomalib.models.image.efficient_ad.torch_model import EfficientAdModelSize
from anomalib.utils.post_processing import superimpose_anomaly_map
from anomalib import TaskType
from anomalib.deploy.inferencers import TorchInferencer, OpenVINOInferencer
from anomalib.metrics import AUROC, AUPRO
from anomalib.loggers import AnomalibTensorBoardLogger
from anomalib.data import Folder
import timeit
import torch
from collections import defaultdict
import os
from IPython.display import clear_output


In [ ]:
from lightning.pytorch import Trainer, seed_everything
seed_everything(42, workers=True)

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


42

In [ ]:
import os

!mkdir -p /content/Delicato_Lavado # Ensure target directory exists
# Check if procitex.zip exists and is valid before unzipping
if not os.path.exists('delicato_lavado.zip'):
    print("Error: delicato_natura.zip not found. Please ensure the zip file is present in the /content directory.")
else:
    # Attempt to unzip to /content, expecting procitex/train etc. inside the zip.
    # If the zip contains a top-level 'procitex' folder, this will result in /content/procitex
    !unzip -o delicato_lavado.zip -d /content/
    print("Unzip command executed. Please verify contents of /content/delicato_lavado")

Archive:  delicato_lavado.zip
  inflating: /content/Delicato_Lavado/test/good/IMG_9478.JPEG  
  inflating: /content/Delicato_Lavado/train/good/IMG_9490.JPEG  
  inflating: /content/Delicato_Lavado/train/good/IMG_9483.JPEG  
  inflating: /content/Delicato_Lavado/train/good/IMG_9479.JPEG  
  inflating: /content/Delicato_Lavado/train/good/IMG_9489.JPEG  
  inflating: /content/Delicato_Lavado/train/good/IMG_9494.JPEG  
  inflating: /content/Delicato_Lavado/train/good/IMG_9501.JPEG  
  inflating: /content/Delicato_Lavado/train/good/IMG_9492.JPEG  
  inflating: /content/Delicato_Lavado/test/good/IMG_9496.JPEG  
  inflating: /content/Delicato_Lavado/test/corte/IMG_9734.JPEG  
  inflating: /content/Delicato_Lavado/train/good/IMG_9481.JPEG  
  inflating: /content/Delicato_Lavado/train/good/IMG_9486.JPEG  
  inflating: /content/Delicato_Lavado/test/corte/IMG_9741.JPEG  
  inflating: /content/Delicato_Lavado/train/good/IMG_9495.JPEG  
  inflating: /content/Delicato_Lavado/test/good/IMG_9493.JPEG 

In [ ]:
root_folder = "Delicato_Lavado"

# Percorre todas as pastas e subpastas
for dirpath, dirnames, filenames in os.walk(root_folder):
    for filename in filenames:
        if filename.endswith(".JPEG"):  # verifica arquivos .JPEG
            old_path = os.path.join(dirpath, filename)
            new_filename = filename.replace(".JPEG", ".jpeg")
            new_path = os.path.join(dirpath, new_filename)
            os.rename(old_path, new_path)  # renomeia inplace

print("Conversão concluída em todas as pastas!")

Conversão concluída em todas as pastas!


In [ ]:
root_path = os.path.abspath("/content/Delicato_Lavado/")

data_module = Folder(
    name="Database/",
    root=root_path,
    normal_dir="train",
    abnormal_dir="test",
    seed=42,
    train_batch_size=1,

)

In [ ]:
data_module.setup()

Show the first image of the dataset


FastFlow model with a Wide-ResNet50-2 backbone was trained using Anomalib, with early stopping and model checkpointing based on AUROC scores.

In [ ]:
TenLogger = AnomalibTensorBoardLogger("logs/Cflow ad/tb_logs", name="Basemodel", log_graph=True)



callbacks = [
    ModelCheckpoint(
        monitor="train_loss",
        mode="min"
    ),
    EarlyStopping(
        monitor="train_loss",
        mode="min",
        patience=5
    )
]


# Setup the datamodule
data_module.setup()

model_base = Cflow(backbone='wide_resnet50_2', pre_trained=True)
engine = Engine( callbacks=callbacks, logger=TenLogger, max_epochs = 10)

# Train the model
engine.fit(model_base, data_module)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
INFO:lightning_fabric.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:lightning_fabric.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name           ┃ Type          ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ pre_processor  │ PreProcessor  │      0 │ train │     0 │
│ 1 │ post_processor │ PostProcessor │      0 │ train │     0 │
│ 2 │ evaluator      │ Evaluator     │      0 │ train │     0 │
│ 3 │ model          │ CflowModel    │  236 M │ train │     0 │
└───┴────────────────┴───────────────┴────────┴───────┴───────┘

Trainable params: 81.6 M                                                                                           
Non-trainable params: 154 M                                                                                        
Total params: 236 M                                                                                                
Total estimated model params size (MB): 946                                                                        
Modules in train mode: 145                                                                                         
Modules in eval mode: 214                                                                                          
Total FLOPs: 0

/usr/local/lib/python3.12/dist-packages/lightning/pytorch/loggers/tensorboard.py:195: Could not log computational graph to TensorBoard: The `model.example_input_array` attribute is not set or `input_array` was not given.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Output()

In [ ]:
import time
import torch
from torchvision import transforms
from PIL import Image
from pathlib import Path

# Caminho do checkpoint salvo após o treino
ckpt_path = "/content/results/Cflow/Database/latest/weights/lightning/model.ckpt"

# Carrega o modelo
model = Cflow.load_from_checkpoint(ckpt_path)
test_root = Path("/content/Delicato_Lavado/test")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

preprocess = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Threshold do modelo
try:
    threshold = model.threshold.value.item()
except Exception:
    threshold = 0.7
print(f"Threshold usado: {threshold:.4f}")

# Função para encontrar imagens de várias extensões
def find_images(folder):
    exts = ["*.png", "*.jpg", "*.jpeg"]
    images = []
    for ext in exts:
        images.extend(folder.glob(ext))
    return images

# ==========================
# Avaliar pasta "good"
# ==========================
good_folder = test_root / "good"
if good_folder.exists():
    good_images = find_images(good_folder)
    correct_good = 0
    total_good = len(good_images)

    total_time = 0.0  # 🔹 acumulador de tempo

    for img_path in good_images:
        image = Image.open(img_path).convert("RGB")
        image_tensor = preprocess(image).unsqueeze(0).to(device)

        torch.cuda.synchronize() if torch.cuda.is_available() else None
        start_time = time.time()

        with torch.no_grad():
            pred = model(image_tensor)
            score = pred.pred_score.cpu().item()

        torch.cuda.synchronize() if torch.cuda.is_available() else None
        total_time += time.time() - start_time

        pred_binary = 1 if score > threshold else 0
        if pred_binary == 0:
            correct_good += 1

    avg_time = total_time / total_good if total_good > 0 else 0
    fps = 1 / avg_time if avg_time > 0 else 0
    percent_correct_good = 100 * correct_good / total_good if total_good > 0 else 0

    print(f"GOOD: {percent_correct_good:.2f}% das imagens boas classificadas corretamente ({correct_good}/{total_good})")
    print(f" Tempo médio por imagem (GOOD): {avg_time*1000:.2f} ms ({fps:.2f} FPS)")
else:
    print(" Pasta 'good' não encontrada!")

# ==========================
# 🔹 Avaliar pastas defeituosas
# ==========================
defective_folders = [f for f in test_root.iterdir() if f.is_dir() and f.name != "good"]

for folder in defective_folders:
    images = find_images(folder)
    if not images:
        print(f" Nenhuma imagem encontrada na pasta {folder.name}")
        continue

    correct_defective = 0
    total_defective = 0
    total_time = 0.0

    for img_path in images:
        image = Image.open(img_path).convert("RGB")
        image_tensor = preprocess(image).unsqueeze(0).to(device)

        torch.cuda.synchronize() if torch.cuda.is_available() else None
        start_time = time.time()

        with torch.no_grad():
            pred = model(image_tensor)
            score = pred.pred_score.cpu().item()

        torch.cuda.synchronize() if torch.cuda.is_available() else None
        total_time += time.time() - start_time

        pred_binary = 1 if score > threshold else 0
        total_defective += 1
        if pred_binary == 1:
            correct_defective += 1

    avg_time = total_time / total_defective if total_defective > 0 else 0
    fps = 1 / avg_time if avg_time > 0 else 0
    percent_correct = 100 * correct_defective / total_defective

    print(f" {folder.name}: {percent_correct:.2f}% das imagens defeituosas detectadas ({correct_defective}/{total_defective})")
    print(f"Tempo médio por imagem ({folder.name}): {avg_time*1000:.2f} ms ({fps:.2f} FPS)")


Threshold usado: 0.7000
GOOD: 60.00% das imagens boas classificadas corretamente (3/5)
 Tempo médio por imagem (GOOD): 119.49 ms (8.37 FPS)
 Nenhum: 64.29% das imagens defeituosas detectadas (9/14)
Tempo médio por imagem (Nenhum): 120.07 ms (8.33 FPS)
 corte: 38.46% das imagens defeituosas detectadas (5/13)
Tempo médio por imagem (corte): 141.66 ms (7.06 FPS)
 costura: 22.22% das imagens defeituosas detectadas (2/9)
Tempo médio por imagem (costura): 113.12 ms (8.84 FPS)


In [ ]:
results = engine.test(model_base, data_module)
print(results)

INFO:lightning_fabric.utilities.rank_zero:The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: Evaluator, ImageVisualizer, PostProcessor, PreProcessor
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

/usr/local/lib/python3.12/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of
metric AUROC was called before the ``update`` method which may lead to errors, as metric states have not yet been 
updated.
  warnings.warn(*args, **kwargs)

/usr/local/lib/python3.12/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of
metric F1Score was called before the ``update`` method which may lead to errors, as metric states have not yet been
updated.
  warnings.warn(*args, **kwargs)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.6052631735801697     │
│       image_F1Score       │    0.9322034120559692     │
└───────────────────────────┴───────────────────────────┘

[{'image_AUROC': 0.6052631735801697, 'image_F1Score': 0.9322034120559692}]
